In [1]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def prophet_forecast_and_evaluate(train_df, target_col, predictor_cols, test_df):
    # Prepare training data for Prophet
    train_df_prophet = train_df.rename(columns={target_col: 'y'})
    train_df_prophet['ds'] = train_df_prophet['datetime']
    train_df_prophet['ds'] = pd.to_datetime(train_df_prophet['ds'])
    
    
    # Initialize Prophet model
    model = Prophet()
    
    # Add additional regressors
    for col in predictor_cols:
        model.add_regressor(col)
    
    # Fit the model
    model.fit(train_df_prophet)
    
    # Cross validation
    df_cv = cross_validation(model, initial='730 days', period='180 days', horizon = '365 days')
    df_p = performance_metrics(df_cv)
    print("Cross-validation Performance Metrics:\n", df_p.head())
    
    # Prepare test data
    test_df_prophet = test_df.copy()
    test_df_prophet['ds'] = test_df_prophet['datetime']
    test_df_prophet['ds'] = pd.to_datetime(test_df_prophet['ds'])
    for col in predictor_cols:
        if col not in test_df_prophet:
            test_df_prophet[col] = np.nan
    
    # Predict
    forecast = model.predict(test_df_prophet.dropna(subset=predictor_cols))
    
    # Fill NaNs in the test DataFrame target column with predictions
    test_df.loc[test_df[target_col].isnull(), target_col] = forecast['yhat'].values
    
    # Visualize the forecast
    fig1 = model.plot(forecast)
    plt.title('Forecast Visualization')
    plt.show()

    # Visualize forecast components
    fig2 = model.plot_components(forecast)
    plt.show()
    
    return test_df

# Example usage (you'll need to replace 'train_df', 'target_col', 'predictor_cols', and 'test_df' with actual variables)
# test_df_with_predictions = prophet_forecast_and_evaluate(train_df, 'target_column_name', ['predictor1', 'predictor2'], test_df)


In [72]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

def adjust_daily_values_enhanced(df, date_col, target_col, date, target_avg):
    # Convert the date_col to datetime format and filter the DataFrame for the given date
    df[date_col] = pd.to_datetime(df[date_col])
    day_data = df[df[date_col].dt.strftime('%m/%d/%Y') == date]
    
    # Calculate original average
    original_avg = day_data[target_col].mean()
    
    # Calculate adjustment factor
    adjustment = target_avg - original_avg
    
    # Adjust the target column values
    day_data['adjusted_' + target_col] = day_data[target_col] + adjustment
    
    # Plot original and adjusted values
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=day_data[date_col], y=day_data[target_col],
                             mode='lines+markers', name='Original',
                             line=dict(color='grey', dash='dash')))
    fig.add_trace(go.Scatter(x=day_data[date_col], y=day_data['adjusted_' + target_col],
                             mode='lines+markers', name='Adjusted',
                             line=dict(color='blue')))
    
    # Enhanced title with original average, target average, and date
    title_text = (f"{target_col.capitalize()} Adjustment for {date}<br>"
                  f"<sup>Original Avg: {original_avg:.2f}, New Avg: {target_avg}")
    
    fig.update_layout(title=title_text,
                      xaxis_title='Hour',
                      yaxis_title=target_col.capitalize(),
                      legend_title=target_col.capitalize(),
                      margin=dict(l=0, r=0, t=50, b=0))  # Adjust top margin to fit enhanced title
    
    fig.show()
    
    return day_data

# Example usage (assuming you have your DataFrame ready)
# df = your_dataframe_here
# adjusted_df = adjust_daily_values_enhanced(df, 'your_date_column', 'your_target_column', 'mm/dd/yyyy', your_target_avg)

